# Day 2 - Lab 2: Documenting Key Decisions with ADRs

**Objective:** Use an LLM as a research assistant to compare technical options and synthesize the findings into a formal, version-controlled Architectural Decision Record (ADR).

**Estimated Time:** 60 minutes

**Introduction:**
Great architectural decisions are based on research and trade-offs. A critical practice for healthy, long-lived projects is documenting *why* these decisions were made. In this lab, you will use an LLM to research a key technical choice for our application and then generate a formal ADR to record that decision for the future.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We'll start by ensuring our environment is ready and adding the standard pathing solution to reliably import our `utils.py` helper.

**Model Selection:**
For research and synthesis tasks, models with large context windows and strong reasoning abilities are ideal. `gpt-4.1`, `gemini-2.5-pro`, or `meta-llama/Llama-3.3-70B-Instruct` would be excellent choices.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the ADR template.
- `save_artifact()`: To save the generated ADR template and the final ADR.

In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact

client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

✅ LLM Client configured: Using 'openai' with model 'gpt-4o'


## Step 2: The Challenges

### Challenge 1 (Foundational): The ADR Template

**Task:** A good ADR follows a consistent format. Your first task is to prompt an LLM to generate a clean, reusable ADR template in markdown.

**Instructions:**
1.  Write a prompt that asks the LLM to generate a markdown template for an Architectural Decision Record.
2.  The template should include sections for: `Title`, `Status` (e.g., Proposed, Accepted, Deprecated), `Context` (the problem or forces at play), `Decision` (the chosen solution), and `Consequences` (the positive and negative results of the decision).
3.  Save the generated template to `templates/adr_template.md`.

In [2]:
# TODO: Write a prompt to generate a markdown ADR template.
adr_template_prompt = """
You are an experienced technical writer specializing in software architecture documentation. Your task is to create a reusable Architectural Decision Record (ADR) template in markdown format. The template should be clean, professional, and adhere to industry best practices.

### Requirements:
1. **Title Section:**
   - Include a placeholder for the ADR title.
   - Example: `# Title: [Decision Title]`

2. **Status Section:**
   - Provide placeholders for status options such as `Proposed`, `Accepted`, or `Deprecated`.
   - Example: `Status: [Proposed | Accepted | Deprecated]`

3. **Context Section:**
   - Include a detailed description of the problem or forces at play.
   - Example: `## Context: [Describe the problem, constraints, and forces influencing the decision.]`

4. **Decision Section:**
   - Provide a clear structure for documenting the chosen solution.
   - Example: `## Decision: [State the decision and its rationale.]`

5. **Consequences Section:**
   - Include placeholders for both positive and negative outcomes of the decision.
   - Example: `## Consequences: [List the expected benefits and potential drawbacks.]`

6. **Additional Notes:**
   - Add a section for any supplementary information or references.
   - Example: `## Notes: [Include any additional context or references.]`

### Output:
- Ensure the template is formatted in valid markdown.
- Use comments or placeholders to guide users on how to fill out each section.
- The output should be clean and ready for immediate use in a version-controlled repository.
"""

print("--- Generating ADR Template ---")
adr_template_content = get_completion(adr_template_prompt, client, model_name, api_provider)
print(adr_template_content)

# Save the artifact
if adr_template_content:
    save_artifact(adr_template_content, "templates/adr_template.md")

--- Generating ADR Template ---
Here's a reusable Architectural Decision Record (ADR) template in markdown format:

```markdown
# Title: [Decision Title]

**Status:** [Proposed | Accepted | Deprecated]

## Context
[Describe the problem, constraints, and forces influencing the decision. Include any relevant background information or business requirements that are driving this decision.]

## Decision
[State the decision clearly and explain the rationale behind it. Describe the solution you have chosen and why it is preferred over other alternatives.]

## Consequences
[List the expected benefits and potential drawbacks of the decision. Discuss both the positive and negative impacts it may have on the system or organization.]

- **Positive Outcomes:** 
  - [Benefit 1]
  - [Benefit 2]
  - [Additional benefits...]

- **Negative Outcomes:**
  - [Drawback 1]
  - [Drawback 2]
  - [Additional drawbacks...]

## Notes
[Include any additional context, references to related documents, or links to fu

### Challenge 2 (Intermediate): AI-Assisted Research

**Task:** Use the LLM to perform unbiased research on a key technical decision for our project: choosing a database for semantic search.

**Instructions:**
1.  Write a prompt instructing the LLM to perform a technical comparison.
2.  Ask it to compare and contrast two technical options: **"Using PostgreSQL with the `pgvector` extension"** versus **"Using a specialized vector database like ChromaDB or FAISS"**.
3.  The prompt should ask for a balanced view for the specific use case of our new hire onboarding tool.
4.  Store the output in a variable for the next step.

> **Tip:** To get a balanced comparison, explicitly ask the LLM to 'act as an unbiased research assistant' and to list the 'pros and cons for each approach.' This prevents the model from simply recommending the more popular option and encourages a more critical analysis.

In [3]:
# TODO: Write a prompt to research database options.
db_research_prompt = """
You are an unbiased research assistant with expertise in database architecture and semantic search technologies. Your task is to provide a balanced technical comparison for a software development team designing a new hire onboarding tool with a semantic search feature.

### Context:
The onboarding tool is intended for a small-to-medium sized enterprise application. The database design must adhere to the following principles:
1. **Scalability:** Ensure the solution can handle increasing data volumes and user queries efficiently.
2. **Modularity:** Support future enhancements, such as integrating additional data sources or features.
3. **TOGAF Principles:** Align with architectural principles of modularity, scalability, and maintainability.
4. **ACID Compliance:** Ensure data integrity, consistency, and reliability.
5. **UML/SysML Compatibility:** Facilitate future diagramming and visualization of the database architecture.

### Task:
Compare and contrast the following two approaches:

1. **Approach 1:** Using PostgreSQL with the `pgvector` extension.
   - Evaluate its suitability for semantic search tasks.
   - Discuss its operational complexity, cost, query flexibility, and scalability.
   - Highlight its strengths and limitations in the context of the onboarding tool.

2. **Approach 2:** Using a specialized, dedicated vector database (e.g., ChromaDB, FAISS, Weaviate).
   - Assess its performance for semantic search use cases.
   - Analyze its operational complexity, cost, query flexibility, and scalability.
   - Identify its advantages and drawbacks for the given use case.

### Output:
Provide a detailed summary of the pros and cons for each approach. Ensure the analysis is unbiased and considers the specific requirements of the onboarding tool. The output should be structured and ready for inclusion in an Architectural Decision Record (ADR).
"""

print("--- Researching Database Options ---")
db_research_output = get_completion(db_research_prompt, client, model_name, api_provider)
print(db_research_output)

--- Researching Database Options ---
### Architectural Decision Record (ADR) - Semantic Search Database Technology

#### Context
The development team is tasked with designing a new hire onboarding tool for a small-to-medium sized enterprise. The tool requires a semantic search feature. The database design must adhere to scalability, modularity, TOGAF principles, ACID compliance, and compatibility with UML/SysML for future diagramming. Two database approaches are under consideration: PostgreSQL with the `pgvector` extension and a specialized vector database.

---

### Approach 1: PostgreSQL with the `pgvector` Extension

#### Suitability for Semantic Search
- **Semantic Search Capability:** `pgvector` enables PostgreSQL to handle vector data, allowing it to perform semantic searches by comparing vector similarities.
- **Integration:** Seamlessly integrates with existing PostgreSQL deployments, making it suitable for teams already using PostgreSQL.

#### Operational Complexity
- **Comple

### Challenge 3 (Advanced): Synthesizing the ADR

**Task:** Provide the LLM with your research from the previous step and have it formally document the decision.

**Instructions:**
1.  Load the `adr_template.md` you created in the first challenge.
2.  Create a new prompt instructing the LLM to act as a Staff Engineer.
3.  Provide the `db_research_output` as context.
4.  Instruct the LLM to populate the ADR template, formally documenting the decision to **use PostgreSQL with pgvector** and justifying the choice based on the synthesized pros and cons.
5.  Save the final, completed ADR as `artifacts/adr_001_database_choice.md`.

In [4]:
adr_template = load_artifact("templates/adr_template.md")

# TODO: Write a prompt to synthesize the final ADR.
synthesis_prompt = f"""
You are a Staff Engineer responsible for documenting key architectural decisions for a new hire onboarding tool. Your task is to populate the provided ADR template to formally document the decision to **use PostgreSQL with the pgvector extension** for the semantic search feature.

### Context:
The onboarding tool is intended for a small-to-medium sized enterprise application. The architectural decision must adhere to the following principles:
1. **Scalability:** Ensure the solution can handle increasing data volumes and user queries efficiently.
2. **Modularity:** Support future enhancements, such as integrating additional data sources or features.
3. **TOGAF Principles:** Align with architectural principles of modularity, scalability, and maintainability.
4. **ACID Compliance:** Ensure data integrity, consistency, and reliability.
5. **UML/SysML Compatibility:** Facilitate future diagramming and visualization of the database architecture.

### Task:
1. Use the provided ADR template to document the decision.
2. Populate the 'Context' section with a summary of the problem, constraints, and forces influencing the decision, based on the research provided.
3. Populate the 'Consequences' section with the expected benefits and potential drawbacks of the decision, ensuring a balanced and objective analysis.
4. Ensure the ADR is complete, professional, and ready for review.

--- ADR TEMPLATE ---
{adr_template}
--- END TEMPLATE ---

--- RESEARCH CONTEXT ---
{db_research_output}
--- END CONTEXT ---

The final ADR should be structured, thorough, and adhere to the specified principles.
"""

print("--- Synthesizing Final ADR ---")
if adr_template and 'db_research_output' in locals() and db_research_output:
    final_adr = get_completion(synthesis_prompt, client, model_name, api_provider)
    print(final_adr)
    save_artifact(final_adr, "artifacts/adr_001_database_choice.md")
else:
    print("Skipping ADR synthesis because template or research is missing.")

--- Synthesizing Final ADR ---
```markdown
# Title: Use PostgreSQL with the pgvector Extension for Semantic Search

**Status:** Accepted

## Context
The new hire onboarding tool for our small-to-medium sized enterprise requires an efficient semantic search feature. The architectural decision must align with key principles: scalability, modularity, TOGAF compliance, ACID compliance, and compatibility with UML/SysML for future diagramming.

Two main approaches were considered: using PostgreSQL with the `pgvector` extension and implementing a specialized vector database. The PostgreSQL option was evaluated for its ability to handle semantic search through vector data comparisons, its seamless integration with existing deployments, and its cost-effective nature due to open-source availability. 

Our choice had to consider the operational complexity, cost implications, query flexibility, and scalability requirements, as well as the need for strong ACID compliance and UML/SysML compatibility

## Lab Conclusion

Well done! You have used an LLM to automate a complex but critical part of the architectural process. You leveraged its vast knowledge base for research and then used it again for synthesis, turning raw analysis into a formal, structured document. This `adr_001_database_choice.md` file now serves as a permanent, valuable record for anyone who works on this project in the future.

> **Key Takeaway:** The pattern of **Research -> Synthesize -> Format** is a powerful workflow. You can use an LLM to gather unstructured information and then use it again to pour that information into a structured template, creating high-quality, consistent documentation with minimal effort.